In [88]:
#Import Libraries
import numpy as np
from nltk import sent_tokenize, word_tokenize
import nltk
import pandas as pd
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.corpus import sentiwordnet as swn
import re
from textblob import TextBlob
import keras


In [89]:
#Lemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

#Lexicon 
stopwords = [ "hmm","ok","th","rd","th","nd","!","#","$","%","&","'","’","u","v","b","r","(",")","*","+","-",".","/",":",";","<","=",">","?","@","[","/","]","^","_","`","{","|","}","~",",","a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", "ourselves", "out", "over", "own", "same", "she","please","sorry","bye", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves","hello"]
greetings = ["hello","good morning","good afternoon","good evening","hi"]
confirm = ["right","yes","correct","absolutely","certainly"]
disagree = ["no","nope"]
apology = ["sorry","apologize"]
gratitude = ["thanks","thank you","nice of you","my pleasure","helpful","thank","no problem"]
agree = ["ok","sure"]
book = ["book"]
without="?"
price = ["how much"]
months=['january','february','march','april','may','june','july','august','september','october','november','december']
cities=["new delhi","london","new york","chennai","bangalore","hyderabad","kochi","seattle","boston","berlin"]
countries=['india','dubai','usa','pakistan','uk','australia','singapore','germany','france','malaysia','japan','china']
states=['tamil nadu','kerala','andhra pradesh','telangana','maharashtra','madhya pradesh','delhi','kerala','karnataka','mumbai','uttar pradesh','west bengal','himachal pradesh','haryana','rajasthan','jharkhand','manipur','meghalaya','sikkim','tripura','assam','chhattisgarh','goa','gujarat','arunachal pradesh','bihar','mizoram','nagaland','odisha','punjab','uttarkhand']

In [90]:
#Read dataset
Dataset = pd.read_fwf('Conversations(Customer)-en.txt',header=None)

In [91]:
#Dataset
list(Dataset)
X = Dataset.iloc[:, 0].values
Dataset

,0,1,2,3
0,"Hello, I would like to book a room for tonight.",NaN,NaN,NaN
1,"Hello, I would like to book a room for Friday,...",NaN,NaN,NaN
2,"Hello, I would like to book a room for the 20th.",NaN,NaN,NaN
3,"Hello, I would like to book a room till the 23rd.",NaN,NaN,NaN
4,"I would like a double room, please.",NaN,NaN,NaN
...,...,...,...,...
152,Sir this is the policy of our hotel and we hav...,NaN,NaN,NaN
153,Sure Sir.,NaN,NaN,NaN
154,This is the best hotels in Dubai,NaN,NaN,NaN
155,These are the Room types available,NaN,NaN,NaN


In [92]:
c=0

#Data Structures - Lists
entities=[]
sentiments=[]
sentences=[]
verbs=[]
subjects=[]
dates=[]
time=[]
Cities=[]
Countries=[]
States=[]
obj=[]
Sentiscore=[]
Score=[]
F=[]
Till=[]
S=[]
intent=[]
Type=[]

for x in X:
    x=str(x)
    
    #Sentence Tokenization 
    sentence = sent_tokenize(x)
    
    for line in sentence:
        
        sentences.append(line)
        
        #Convert to lowercase
        line = line.lower()
        line = str(line)
        
        #Polarity Calculation 
        s = TextBlob(line)
        S.append(s.sentiment.polarity)
        if s.sentiment.polarity > 0:
           Sentiscore.append("Positive")
        elif s.sentiment.polarity < 0:
           Sentiscore.append("Negative")
        elif s.sentiment.polarity == 0:
           Sentiscore.append("Neutral")
        
        #Copy
        copy = line
        word = nltk.word_tokenize(copy)
        
        #Request without data
        U=False
        if "?" not in line:
            U = True
        
        #Remove Digits 
        line = ''.join(c for c in line if not c.isdigit())
        
        #Type
        if '!' in line:
            types = "Exclamation"
        elif '?' in line:
            types = "Question"
        else:
            types = "Statement"
        Type.append(types)
        
        #Intent
        G = list()
        for g in greetings:
            if g in line:
                G.append("Greeting")
        for c in confirm:
            if c in line:
                G.append("Confirmation")
        for A in agree:
            if A in line:
                G.append("Agreement")
        for b in book:
            if b in line:
                G.append("Request to book")
        for p in price:
            if p in line:
                G.append("Request for pricing")
        if without in line:
            G.append("Request with missing data") 
        for ap in apology:
            if ap in line:
                G.append("Apologetic")
        for gr in gratitude:
            if ap in line:
                G.append("Gratitude")
        for di in disagree:
            if ap in line:
                G.append("Disagreement")
        
        #Regular Expression 
        
        #From and To Date
        date = re.search("([0-9][0-9]th|[0-9][0-9]nd|[0-9][0-9]st|[0-9][0-9]rd)", copy)    
        f = re.search("(for.*[0-9][0-9]th|for.*[0-9][0-9]nd|for.*[0-9][0-9]st|for.*[0-9][0-9]rd)", copy) 
        if f:
            f = re.search("([0-9][0-9]th|[0-9][0-9]nd|[0-9][0-9]st|[0-9][0-9]rd)", f.group())  
            f = f.group()
        else: 
            f = "None"
        till = re.search("(till.*[0-9][0-9]th|till.*[0-9][0-9]nd|till.*[0-9][0-9]st|till.*[0-9][0-9]rd)", copy)    
        if till:
            till = re.search("([0-9][0-9]th|[0-9][0-9]nd|[0-9][0-9]st|[0-9][0-9]rd)", till.group()) 
            till = till.group()
        else:
            till = "None"
        
        #Word Tokenization 
        token= nltk.word_tokenize(line)
        
        #StopWord Removal 
        token = [words for words in token if words not in stopwords]
                    
        #Find Duration
        w=""
        for h in word:
                if h=="days":
                    o=word.index(h)
                    w=word[o-1]+" "+h
                elif h=="weeks":
                    o=word.index(h)
                    w=word[o-1]+" "+h
        if w=="":
            w="None"
        w=str(w)
        time.append(w)
        w=""
        
        #Find Location 
        j=""
        for h in word:
            for r in cities:
                if h==r:
                    j=h
        if j=="":
            j="None"
        Cities.append(j)
        co=""
        for h in word:
            for r in countries:
                if h==r:
                    co=h
        if co=="":
            co="None"
        Countries.append(co)
        st=""
        for h in word:
            for r in states:
                if h==r:
                    st=h
        if st=="":
            st="None"
        States.append(st)
        
        #Lemmatization 
        token = [wordnet_lemmatizer.lemmatize(word) for word in token]
        
        #POS Tagger
        tags = nltk.pos_tag(token)
        Tags = nltk.pos_tag(word)
        length = len(tags)
        lc = len(Tags)
        
        #Find Date
        mon=""
        for h in word:
            for r in months:
                if h==r:
                    mon=h
                    mon=str(mon)
        if mon and f:
            if f!="None":
                f=f+" "+mon
        if mon and till:
            if till!="None":
                till=till+" "+mon
        Till.append(till)
        F.append(f)
        
        if date and mon:
            m = date.group()
            m=str(m)
            u = m+" "+mon;
        elif date:
            m = date.group()
            u = m;
        elif mon:
            u = mon;
        else:
            u="None"   
        dates.append(u)
        m=""
        
        
        a = list()
        b = list()
        d = list()
        e = list()
        
        #Entity Identification
        for i in range(0, lc):
          log = (Tags [i][1] == 'NN')
          if log == True:
             a.append(Tags [i][0])
        for i in range(0, lc):
          log = (Tags [i][1] == 'NNS')
          if log == True:
             a.append(Tags [i][0])
        
        #Request with data
        for i in range(0, lc):
          log = (Tags [i][1] == 'MD')
          if log == True and U==True:
             G.append("Request with data")
            
        #Subject Detection 
        for i in range(0, lc):
          log = (Tags [i][1] == 'NN')
          if log == True:
             e.append(Tags [i][0])
             break;
    
    
        y=0     
        
        #Negation Resolution 
                
        #Verb Detection 
        for i in range(0, length):
            if(tags[i][0]=="not"):
                y=1
            log = (tags [i][1] == 'VB')
            if log == True and y==1:
                d.append("not "+tags [i][0])
            elif log== True and y!=1:
                d.append(tags[i][0])        
        
        #Object Extraction 
        l=a;
        l=[words for words in l if words not in e]
        
        #StopWord Removal 
        l = [words for words in l if words not in stopwords]
        a = [words for words in a if words not in stopwords]
        e = [words for words in e if words not in stopwords]
        
        #Final Columns
        Score.append(f)
        Tags.append(tags)
        entities.append(a)
        sentiments.append(b)
        verbs.append(d)
        subjects.append(e)
        obj.append(l)
        intent.append(G)



In [93]:
#Creating Dataframe 
data = {'Sentence':sentences,'Type':Type,'Entities':entities,'Intent of the user':intent,'Subject':subjects,'Objects':obj,'Verb':verbs, 'Sentiment Polarity':Sentiscore,"Sentiment Score":S, 'Date':dates, 'Country':Countries,"State":States,'City':Cities, "From Date":F,"To Date":Till,"Duration":time} 
data = pd.DataFrame(data)
data


,Sentence,Type,Entities,Intent of the user,Subject,Objects,Verb,Sentiment Polarity,Sentiment Score,Date,Country,State,City,From Date,To Date,Duration
0,"Hello, I would like to book a room for tonight.",Statement,"[book, room, tonight]","[Greeting, Agreement, Request to book, Request...",[],"[book, room, tonight]",[],Neutral,0.0,None,None,None,None,None,None,None
1,"Hello, I would like to book a room for Friday,...",Statement,"[book, room, friday]","[Greeting, Agreement, Request to book, Request...",[],"[book, room, friday]",[],Neutral,0.0,15th august,None,None,None,15th august,None,None
2,"Hello, I would like to book a room for the 20th.",Statement,"[book, room]","[Greeting, Agreement, Request to book, Request...",[],"[book, room]",[],Neutral,0.0,20th,None,None,None,20th,None,None
3,"Hello, I would like to book a room till the 23rd.",Statement,"[book, room]","[Greeting, Agreement, Request to book, Request...",[],"[book, room]",[],Neutral,0.0,23rd,None,None,None,None,23rd,None
4,"I would like a double room, please.",Statement,[room],[Request with data],[],[room],[],Neutral,0.0,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241,Sir this is the policy of our hotel and we hav...,Statement,"[sir, policy, hotel]",[Greeting],[sir],"[policy, hotel]",[],Neutral,0.0,None,None,None,None,None,None,None
242,Sure Sir.,Statement,[sir],[Agreement],[sir],[],[],Positive,0.5,None,None,None,None,None,None,None
243,This is the best hotels in Dubai,Statement,"[dubai, hotels]",[Greeting],[dubai],[hotels],[],Positive,1.0,None,dubai,None,None,None,None,None
244,These are the Room types available,Statement,[room],[],[room],[],[],Positive,0.4,None,None,None,None,None,None,None


In [94]:
#Converting to CSV file 
data.to_csv("NLP Problem (3).csv")